<p align="center">
  <img src="https://github.com/sominwadhwa/sominwadhwa.github.io/blob/master/assets/vqa/6.jpg?raw=true"/>
</p>

In [ ]:
import sys
from random import shuffle, sample
import pickle as pk
import gc

import numpy as np
import pandas as pd
import scipy.io
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Reshape
from keras.layers.recurrent import LSTM
from keras.layers import Merge
from keras.layers.merge import Concatenate
from keras.optimizers import SGD
from keras.utils import np_utils, generic_utils
from progressbar import Bar, ETA, Percentage, ProgressBar    
from keras.models import model_from_json
from sklearn.preprocessing import LabelEncoder
import spacy
#from spacy.en import English

from src.utils import freq_answers, grouped, get_questions_sum, get_images_matrix, get_answers_sum
from src.extract_features import get_questions_matrix_sum, get_images_matrix, get_answers_matrix

/Users/sw/.venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
training_questions = open("preprocessed/v2/ques_train.txt","rb").read().decode('utf8').splitlines()
training_questions_len = open("preprocessed/v2/ques_train_len.txt","rb").read().decode('utf8').splitlines()
answers_train = open("preprocessed/v2/answer_train.txt","rb").read().decode('utf8').splitlines()
images_train = open("preprocessed/v2/images_coco_id.txt","rb").read().decode('utf8').splitlines()
img_ids = open('preprocessed/v2/coco_vgg_IDMap.txt').read().splitlines()
vgg_path = "data/coco/vgg_feats.mat"

In [ ]:
sample(list(zip(images_train, training_questions, answers_train)), 5)

In [ ]:
nlp = spacy.load("en_core_web_md")
print ("Loaded WordVec")

In [ ]:
vgg_features = scipy.io.loadmat(vgg_path)
img_features = vgg_features['feats']
id_map = dict()
print ("Loaded VGG Weights")

In [ ]:
gc.collect()

In [ ]:
upper_lim = 1000 #Number of most frequently occurring answers in COCOVQA (Covering >80% of the total data)
training_questions, answers_train, images_train = freq_answers(training_questions, 
                                                               answers_train, images_train, upper_lim)
training_questions_len, training_questions, answers_train, images_train = (list(t) for t in zip(*sorted(zip(training_questions_len, 
                                                                                                          training_questions, answers_train, 
                                                                                                          images_train))))
print (len(training_questions), len(answers_train),len(images_train))

In [ ]:
lbl = LabelEncoder()
lbl.fit(answers_train)
nb_classes = len(list(lbl.classes_))
pk.dump(lbl, open('preprocessed/v2/label_encoder_lstm.sav','wb'))

In [ ]:
batch_size               =      128
img_dim                  =     4096
word2vec_dim             =      300
max_len                  =       30 #Max len of textual vars

num_hidden_nodes_mlp     =     1024
num_hidden_nodes_lstm    =      512
num_layers_mlp           =        3
num_layers_lstm          =        3
dropout                  =       0.5
activation_mlp           =     'tanh'


In [ ]:
# Change the following based on your usage, THESE WILL DIRECTLY AFFECT THE DURATION OF NETWORK TRAINING
num_epochs               =         1 
model_save_interval      =         1 

In [ ]:
image_model = Sequential()
image_model.add(Reshape(input_shape = (img_dim,), dims=(img_dim,)))

In [ ]:
language_model = Sequential()
